In [1]:
# import necessary libraries
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import inspect, create_engine, func, MetaData, Table
import datetime as dt
from datetime import time
import pandas as pd
import numpy as np
import json

In [2]:
# Database Setup
engine = create_engine('sqlite:///SevereWeather.sqlite', echo=False)

# produce our own MetaData object
metadata = MetaData()

# we can reflect it ourselves from a database, using options
# such as 'only' to limit what tables we look at...
metadata.reflect(engine)

Base = automap_base()
Base.prepare(engine, reflect=True)

In [3]:
inspector = inspect(engine)
Events = Table('Events',metadata)
inspector.reflecttable(Events, None)
session = Session(bind=engine)

In [4]:
events_df = pd.read_sql_table('Events', engine)

In [5]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197287 entries, 0 to 197286
Data columns (total 15 columns):
level_0      197287 non-null int64
index        197287 non-null int64
yr           197287 non-null int64
mo           197287 non-null int64
dy           197287 non-null int64
st           197287 non-null object
mag          197287 non-null float64
inj          197287 non-null int64
fat          197287 non-null int64
loss         197287 non-null float64
closs        197287 non-null float64
slat         197287 non-null float64
slon         197287 non-null float64
type         197287 non-null object
date_time    197287 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(7), object(2)
memory usage: 22.6+ MB


In [6]:
tornado_df = events_df.loc[events_df['type']=="torn"]

In [7]:
value = ['2010']
tornado_2010_df = tornado_df[tornado_df.yr.isin(value)]

In [8]:
tornado_2010_df['date_time'] = [dt.date.strftime(x,'%Y-%m-%d%H:%M:%S') for x in tornado_2010_df.date_time]

/Users/tom/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
tornado_2010_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1315 entries, 0 to 1314
Data columns (total 15 columns):
level_0      1315 non-null int64
index        1315 non-null int64
yr           1315 non-null int64
mo           1315 non-null int64
dy           1315 non-null int64
st           1315 non-null object
mag          1315 non-null float64
inj          1315 non-null int64
fat          1315 non-null int64
loss         1315 non-null float64
closs        1315 non-null float64
slat         1315 non-null float64
slon         1315 non-null float64
type         1315 non-null object
date_time    1315 non-null object
dtypes: float64(5), int64(7), object(3)
memory usage: 164.4+ KB


In [ ]:
locations = tornado_2010_df[['slat', 'slon']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[1:5]

In [10]:
from geojson import Feature, FeatureCollection, Point

In [11]:
# columns used for constructing geojson object
features = tornado_2010_df.apply(
    lambda row: Feature(geometry=Point((float(row['slon']), float(row['slat'])))),
    axis=1).values.tolist()

# all the other columns used as properties
properties = tornado_2010_df.drop(['slat', 'slon'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [12]:
with open('tornado_2010.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)